## Import the required Libraries

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# from skmultilearn.adapt import MLkNN
import sklearn.metrics as metrics
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
import os
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, concatenate

2024-04-29 10:15:52.851801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 10:15:52.851834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 10:15:52.852899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 10:15:52.858686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 10:15:53.565124: W tensorflow/comp

## Read the data

In [5]:
df = pd.read_csv("/home/yadagiri/train_dataset.csv")

To reduce the load(on my system), taking first 1087 rows of human and last 1087 rows of gemini-pro. Dropping the rest (We will be considering the entire data set on the PC)

# Drop 'id' and 'category' column

In [6]:
df = df.drop(columns=['id', 'Category'])

In [7]:
df

,text,Vocabulary,Noun Count,Verb Count,AUX Count,NUM Count,PRON Count,ADV Count,INTJ Count,PART Count,label
0,Unveiling the Plot: Evidence Points to Premedi...,279,146,86,2,2,9,13,0,0,1
1,Chris Wallace Departs Fox News After 18 Years ...,215,85,48,1,5,12,5,0,1,1
2,Tiger Woods' Resurgence: A Journey from Scanda...,255,107,64,4,10,31,13,0,0,1
3,Fact Check: Biden Inauguration Amidst Pandemic...,282,130,84,0,6,7,9,0,1,1
4,"Hurricane Ida, the third major hurricane to hi...",114,66,58,2,11,4,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
12145,"Tom Brady Retires, Baker Mayfield Takes Over: ...",261,121,64,10,25,9,10,0,1,1
12146,"NEW YORK (CNN) -- On the morning of Sept. 11, ...",125,37,36,0,4,17,7,0,1,1
12147,Tiger Woods was involved in a serious car cras...,147,103,86,1,1,32,16,0,1,1
12148,Facebook warns of slower revenue growth amid f...,233,106,54,1,17,10,10,0,2,0


# Convert into a label encoder, 0 for 'is_human' and 1 for 'machine'

In [8]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'label' is the column you want to encode
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [9]:
df

,text,Vocabulary,Noun Count,Verb Count,AUX Count,NUM Count,PRON Count,ADV Count,INTJ Count,PART Count,label
0,Unveiling the Plot: Evidence Points to Premedi...,279,146,86,2,2,9,13,0,0,1
1,Chris Wallace Departs Fox News After 18 Years ...,215,85,48,1,5,12,5,0,1,1
2,Tiger Woods' Resurgence: A Journey from Scanda...,255,107,64,4,10,31,13,0,0,1
3,Fact Check: Biden Inauguration Amidst Pandemic...,282,130,84,0,6,7,9,0,1,1
4,"Hurricane Ida, the third major hurricane to hi...",114,66,58,2,11,4,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
12145,"Tom Brady Retires, Baker Mayfield Takes Over: ...",261,121,64,10,25,9,10,0,1,1
12146,"NEW YORK (CNN) -- On the morning of Sept. 11, ...",125,37,36,0,4,17,7,0,1,1
12147,Tiger Woods was involved in a serious car cras...,147,103,86,1,1,32,16,0,1,1
12148,Facebook warns of slower revenue growth amid f...,233,106,54,1,17,10,10,0,2,0


# Store the list of 'text' in X and 'label' in y_data

In [10]:
X = list(df['text'])
y_data = df['label']
y = y_data.values

# Convert into text tokenization

Assuming X is a list of mixed data-types, filter out non-string values

In [11]:
X = [str(x) if isinstance(x, str) else '' for x in X]
len(X)

12150

In [12]:
#Model CNN & Bi-LSTM
import keras
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM
# from keras.layers.recurrent import LSTM
# from keras.layers.core import Activation, Dropout, Dense
from keras.layers import LSTM, Activation, Dropout, Dense, Flatten, Bidirectional, GRU, concatenate, SpatialDropout1D

from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional,GRU,concatenate,SpatialDropout1D
from keras.layers import GlobalMaxPooling1D,GlobalAveragePooling1D,Conv1D
from keras.models import Model
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate
import matplotlib.pyplot as plt
from keras import layers
from keras.optimizers import Adam,SGD,RMSprop
from keras.losses import MeanSquaredError
######## Textual Features for Embedding ###################
max_len = 100
max_features = 10000
# max_features = 4479
embed_size = 300
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, padding='post', maxlen=max_len)

In [13]:
X

array([[4702,  562,   24, ...,    9,  105,  515],
       [1934,  415, 6423, ...,    1,  773,  655],
       [3554,    2, 4050, ..., 1092, 1295, 3748],
       ...,
       [4822,    3,   18, ...,    6,    1, 1149],
       [ 386,   11, 3389, ...,    6,    5,  172],
       [   4,    1,  240, ...,   30,   28,  477]], dtype=int32)

In [14]:
y

array([1, 1, 1, ..., 1, 0, 1])

In [15]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=2)
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [16]:
len(y)

12150

# Defining the Embedding Matrix

In [17]:
from numpy import array
from numpy import asarray
from numpy import zeros
# Load GloVe word embeddings
embeddings_dictionary = {}
embedding_dim = 100  # Adjust according to the dimensions of your GloVe embeddings file

glove_file = open("/home/yadagiri/glove.6B.300d.txt")  # Update the path
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

vocab_size = len(tokenizer.word_index) + 1  ## total distinct words is the Vocabulary ##
word_index = tokenizer.word_index
num_words = min(max_features,len(word_index)+1)
embed_size = 300
embedding_matrix = zeros((num_words, embed_size)) ## has to be similar to glove dimension ##
for word, index in tokenizer.word_index.items():
    if index >= max_features:
        continue
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [18]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
        -0.20988999,  0.053913  ],
       [ 0.038466  , -0.039792  ,  0.082747  , ..., -0.33427   ,
         0.011807  ,  0.059703  ],
       ...,
       [-0.04136   , -0.064987  , -0.19898   , ...,  0.33342999,
        -0.17115   , -0.080621  ],
       [-0.042367  , -0.18526   ,  0.37704   , ...,  0.65853   ,
        -0.28538001, -0.052693  ],
       [ 0.20338   ,  0.15666001,  0.33083001, ...,  0.1302    ,
        -0.074687  , -0.55899   ]])

# Load the text as well as numerical data, then preprocess it

In [19]:
texts = df['text'].tolist()

# texts = X
# texts = texts.tolist()
numerical_data = df[['Vocabulary', 'Noun Count', 'Verb Count', 'AUX Count', 'NUM Count', 'PRON Count', 'ADV Count', 'INTJ Count', 'PART Count']].values

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
max_sequence_length = max([len(seq) for seq in sequences])
data = pad_sequences(sequences, maxlen=max_sequence_length)

# # Normalize numerical data if needed
# numerical_data_normalized = (numerical_data - numerical_data.mean(axis=0)) / numerical_data.std(axis=0)

X = np.concatenate((data, numerical_data), axis=1)
y = df['label'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
X.shape

(12150, 1269)

In [21]:
y.shape

(12150,)

In [22]:
print(embedding_matrix.shape)

(10000, 300)


In [23]:
print(vocab_size, embedding_dim)

36609 100


# Define and build the model

In [24]:
vocab_size = len(word_index) + 1
embedding_dim = embedding_matrix.shape[1]
num_numerical_features = numerical_data.shape[1]
num_classes = len(np.unique(y))
embedding_matrix = np.zeros((vocab_size, embedding_dim))

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))

# Adding numerical data as input
numerical_input = Input(shape=(num_numerical_features,))
numerical_dense = Dense(32, activation='relu')(numerical_input)

# Concatenate text and numerical features
concatenated = concatenate([model.output, numerical_dense])

# Final output layer
output = Dense(num_classes, activation='softmax')(concatenated)

final_model = Model(inputs=[model.input, numerical_input], outputs=output)
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


2024-04-29 10:16:23.820459: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-29 10:16:23.820492: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: nits-lab
2024-04-29 10:16:23.820501: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: nits-lab
2024-04-29 10:16:23.820676: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.171.4
2024-04-29 10:16:23.820705: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.171.4
2024-04-29 10:16:23.820711: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.171.4


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
X_train.shape

(9720, 1269)

In [27]:
X_test.shape

(2430, 1269)

In [28]:
y.shape

(12150,)

# Train the model

In [29]:
final_model.fit([X_train[:, :max_sequence_length], X_train[:, max_sequence_length:]], y_train, epochs=10, batch_size=128, validation_data=([X_test[:, :max_sequence_length], X_test[:, max_sequence_length:]], y_test))

Epoch 1/10
76/76 [==============================] - 41s 494ms/step - loss: 0.6816 - accuracy: 0.8913 - val_loss: 0.3588 - val_accuracy: 0.9189
Epoch 2/10
76/76 [==============================] - 37s 487ms/step - loss: 0.2862 - accuracy: 0.9214 - val_loss: 0.2706 - val_accuracy: 0.9214
Epoch 3/10
76/76 [==============================] - 37s 484ms/step - loss: 0.2502 - accuracy: 0.9267 - val_loss: 0.2532 - val_accuracy: 0.9206
Epoch 4/10
76/76 [==============================] - 37s 483ms/step - loss: 0.2355 - accuracy: 0.9289 - val_loss: 0.2663 - val_accuracy: 0.9218
Epoch 5/10
76/76 [==============================] - 37s 482ms/step - loss: 0.2303 - accuracy: 0.9285 - val_loss: 0.2408 - val_accuracy: 0.9280
Epoch 6/10
76/76 [==============================] - 37s 486ms/step - loss: 0.2159 - accuracy: 0.9329 - val_loss: 0.2368 - val_accuracy: 0.9235
Epoch 7/10
76/76 [==============================] - 37s 485ms/step - loss: 0.2185 - accuracy: 0.9335 - val_loss: 0.2260 - val_accuracy: 0.9255

# Evaluate the model

In [30]:
# Evaluate the model on the test set
loss, accuracy = final_model.evaluate([X_test[:, :max_sequence_length], X_test[:, max_sequence_length:]], y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Calculate additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = final_model.predict([X_test[:, :max_sequence_length], X_test[:, max_sequence_length:]])
y_pred_classes = np.argmax(y_pred, axis=1)

precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


76/76 [==============================] - 4s 58ms/step - loss: 0.2482 - accuracy: 0.9169
Test Accuracy: 91.69%
76/76 [==============================] - 5s 54ms/step
Precision: 0.9112
Recall: 0.9169
F1 Score: 0.9138


In [31]:
# Filter predictions and true labels for "AI" and "human" classes
ai_human_mask = np.isin(y_test, ['AI', 'human'])

# Debugging code to inspect values
print("Filtered Mask:", ai_human_mask)
print("Filtered y_test:", y_test[ai_human_mask])

y_test_filtered = y_test[ai_human_mask]
y_pred_classes_filtered = y_pred_classes[ai_human_mask]

# Generate classification report for "AI" and "human" classes
from sklearn.metrics import classification_report
report = classification_report(y_test_filtered, y_pred_classes_filtered)

# Print the classification report
print(report)


Filtered Mask: [False False False ... False False False]
Filtered y_test: []


/home/yadagiri/anaconda3/lib/python3.11/site-packages/numpy/lib/arraysetops.py:733: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: max() arg is an empty sequence